# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [34]:
SELECT * FROM FCID_Description;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [7]:
CREATE VIEW Metricas AS
SELECT count(distinct SEQN) as popularidade, SUM(intake) as Intake_Sum, AVG(intake) as Intake_AVG, AVG(intake_bw) as Intake_AVG_BW, I.FCID_CODE, contagem
FROM Intake I
JOIN
(SELECT FCID_CODE, COUNT(*) as contagem
FROM Recipes
GROUP BY FCID_CODE) as R
ON I.FCID_CODE = R.FCID_CODE
GROUP BY I.FCID_CODE;

In [8]:
SELECT * FROM Metricas LIMIT 10

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [9]:
SELECT M.*, F.FCID_DESC FROM Metricas M
JOIN FCID_Description F ON 
M.FCID_CODE = F.FCID_CODE
ORDER BY intake_sum DESC 
LIMIT 20

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

Podemos contruir uma view com a seguintes métricas:

| Métrica | Descrição |
| --- | --- |
| Intake_Sum | total consumido do alimento em gramas por pessoa |
| Intake_AVG | média de consumo do alimento em gramas por pessoa |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |


A partir destas métricas, é possível obter quais são os 10 alimentos mais consumidos por pessoa. Utilizando o CGN de cada um dos 10 alimentos, podemos gerar um grupo a qual essa pessoa pertence que é a concatenação dos CGN dos alimentos.

In [41]:
DROP VIEW IF EXISTS Metrica_Pessoa_Top10;
DROP VIEW IF EXISTS Metrica_Pessoa;

CREATE VIEW Metrica_Pessoa as
SELECT SEQN, FCID_CODE, SUM(intake) as intake_sum, AVG(intake) as intake_avg, AVG(intake_bw) as INTAKE_BW_AVG
FROM Intake
Group BY SEQN, FCID_CODE
ORDER BY INTAKE_SUM DESC;


CREATE VIEW Metrica_Pessoa_Top10 as
SELECT m1.SEQN, m1.FCID_CODE, m1.intake_sum, m1.intake_avg, m1.INTAKE_BW_AVG, F.FCID_DESC, F.CGN
FROM Metrica_Pessoa m1
JOIN FCID_DESCRIPTION F
on m1.FCID_CODE = F.FCID_CODE
WHERE F.CGN IS NOT 86 AND
(
    SELECT COUNT(*) 
    FROM Metrica_Pessoa m2 
    WHERE m2.SEQN = m1.SEQN AND m2.intake_sum > m1.intake_sum
) < 5
ORDER BY m1.SEQN, m1.intake_sum DESC;



In [42]:
SELECT * FROM Metrica_Pessoa_Top10

In [50]:
DROP VIEW IF EXISTS Metrica_Pessoa_Grupo;

CREATE VIEW Metrica_Pessoa_Grupo as
SELECT SEQN, GROUP_CONCAT(CGN) as grupo FROM
(SELECT SEQN, CGN 
 FROM Metrica_Pessoa_Top10
 ORDER BY CGN DESC
)
GROUP BY SEQN;

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [52]:
SELECT count(distinct grupo) from Metrica_Pessoa_Grupo

803

In [51]:
SELECT M.grupo, sum(I.intake) as Intake_Sum, avg(I.intake) as Intake_AVG, avg(I.intake_bw) as Intake_AVG_BW 
FROM Metrica_Pessoa_Grupo M
JOIN Intake I
ON I.SEQN = M.SEQN
GROUP BY M.grupo
ORDER BY INTAKE_SUM DESC
LIMIT 10